# Creative extension analysis notebook

---

**Authors**

- Jérémy Bensoussan
- Ekaterina Kryukova
- Jules Triomphe

---

## Abstract

While the paper examines the exposure hypothesis for all topics, we propose to analyze and compare political and food related tweets. To do so, we plan to obtain egos, alters and their timelines from Twitter’s API by generating $3\times30,000$ random numbers in a range from $0$ to $3,000,000,000$. Then, we intend to identify egos’ retweets using official RT, classify tweets by topics based on hashtags (and keywords if the dataset is lacking content) and create a follower/followee graph. To calculate the probability of retweeting alters’ tweets by egos we will use a more solid approach then what is described in the paper “Differences in the Mechanics of Information Diffusion Across Topics” where the probability is equal to the number of users that were k times exposed to a hashtag and retweeted before the ($k+1$)-th exposure, divided by the number of users that were k-times exposed to the hashtag. Finally, we plan to visualize results as well as analyze the probability by breaking down users based on betweenness, clustering coefficient and number of followees.

## Research Questions

1. Is there a significant difference between the probability of retweeting when the tweet is about food and when it is about politics ?
2. Is there a significant difference in the number of times a tweet is retweeted depending on whether it is about food or politics ?
3. Is there a relation between user betweenness, size of cluster or number of friends with the retweet probabilities ?

## Proposed dataset

Self-collected (with the Twitter API) ego and alter timelines with all tweet fields from the [**GET /2/users** endpoint](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users) and all user fields except for `profile_image_url`.

## Methods

### Data collection

We will sign up for Twitter’s API to collect data. We will generate $3\times30,000$ random numbers in a range from $0$ to $3,000,000,000$ as in the paper and use the [**GET /2/users** endpoint](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users) to collect active and public user information with all tweet fields and all user fields except for profile_image_url.

### Building the network

We will use networkx to build a directed network of followers  in which nodes are users (egos and alters) and edges are the following relationships (without the following relationships among alters). Next, we will build another network where relationships among alters of active egos are included.

### Calculating retweet probability

For each ego, we will count the number of followees who have retweeted a post (exposures) on a certain topic at a certain date. We will get the information like this: an ego $i$ was exposed to $200$ posts about this topic only once, among which $i$ retweeted $50$ (probability is $50/200 = 25\%$); at the same time, $i$ was exposed to $100$ posts about this topic twice, among which $i$ retweeted $50$ ($probability = 50\%$); … Finally, we will calculate a sequence of probability for each ego. (Same procedure as in the paper.) If there is sufficient data, we will apply a t-test to identify whether the distribution of retweets for each exposure count is significantly different from one topic to the other. We will also try to compare the distribution of retweet probabilities between topics.

### Community detection

We will use the second ego networks to compute clustering coefficients and betweenness of the active egos.

### Data analysis

We will compare the retweet probabilities based on betweenness, number of followers and clustering for each topic, much like in *Figure 6* in the paper.

---

## Initialisation

Import modules.

In [1]:
# Import libraries
import random
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm, trange
import requests
import time
from datetime import datetime
import os

<ipython-input-1-21c4b87703f5>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Setup automatic formatting (requires the `nb-black` package).

In [2]:
# Enable auto-formatting

%load_ext lab_black

### Control center

**This is the control center. All operations are decided here to avoid memory overflow and excessive computation times. This notebook should be run FROM THE TOP once these parameters have been set.** If in doubt, ask Jules ;)

In [3]:
# Define constants

# UserID range
LOWER_ID_N = 0
UPPER_ID_N = int(3e9)

# UserID number
N_UID_PER_REQUEST = int(3e4)
N_UID_REQUESTS = 3

# --------------------------------------------------

# Choose whether to generate new UserIDs
CREATE_NEW_UIDs = False

# Choose whether to collect user data
COLLECT_USER_DATA = False
# Select the batch to query if collecting user data
REQUEST_NUMBER = 2
# Define behaviour depending on the run number.
# If this is True then COLLECT_USER_DATA must be True
FIRST_RUN = False

# Chooser whether to create user subset files
CREATE_USER_SUBSETS = True

# Choose whether to create/reset data pull status
CREATE_DATA_PULL_STATUS = True
# Choose whether to pull new data and save it
PULL_NEW_TIMELINE_DATA = True
SAVE_NEW_TIMELINE_DATA = True
PULL_NEW_FOLLOWER_DATA = True
SAVE_NEW_FOLLOWER_DATA = True
PULL_NEW_FRIEND_DATA = True
SAVE_NEW_FRIEND_DATA = True
# Choose whether to save newly pulled data
SAVE_PULLED_DATA = True

# --------------------------------------------------

# Data folder location
DATA_FOLDER = "./data/"
# UIDs
UIDS_FILE = DATA_FOLDER + "uids.csv"
# User files
USERS_FOLDER = DATA_FOLDER + "users/"
USERS_FILE = USERS_FOLDER + "users.csv"
PUBLIC_USERS_FILE = USERS_FOLDER + "public_users.csv"
PUBLIC_USERS_W_TWEETS_FILE = USERS_FOLDER + "public_users_w_tweets.csv"
PUBLIC_USERS_W_FOLLOWERS_FILE = USERS_FOLDER + "public_users_w_followers.csv"
PUBLIC_USERS_W_FRIENDS_FILE = USERS_FOLDER + "public_users_w_friends.csv"
# Pulled data
PUBLIC_USERS_PULL_STATUS_FILE = DATA_FOLDER + "public_users_pull_status.csv"
# Timeline files
TIMELINES_FOLDER = DATA_FOLDER + "timelines/"
PUBLIC_USERS_TIMELINES_FILE = TIMELINES_FOLDER + "public_users_timelines.csv"
# Network
NETWORK_FOLDER = DATA_FOLDER + "network/"
PUBLIC_USERS_FOLLOWERS_FILE = NETWORK_FOLDER + "public_users_followers.csv"
PUBLIC_USERS_FRIENDS_FILE = NETWORK_FOLDER + "public_users_friends.csv"
# File containing the bearer token
BEARER_TOKEN = DATA_FOLDER + "bearer_token.auth"

# API endpoints
API_USERS_ENDPOINT = "https://api.twitter.com/2/users?ids="
API_USER_FIELDS = "user.fields=created_at,description,entities,id,location,name,pinned_tweet_id,protected,public_metrics,url,username,verified,withheld"
API_TWEET_FIELDS = "tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,non_public_metrics,public_metrics,organic_metrics,promoted_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld"
API_V1_RATE_LIMITS = "https://api.twitter.com/1.1/application/rate_limit_status.json?resources=application,statuses,followers,friends"
API_USER_TIMELINE_ENDPOINT = "https://api.twitter.com/1.1/statuses/user_timeline.json"
API_FOLLOWERS_IDS_ENDPOINT = "https://api.twitter.com/1.1/followers/ids.json"
API_FRIENDS_IDS_ENDPOINT = "https://api.twitter.com/1.1/friends/ids.json"

# Random seed
SEED = 30
random.seed(SEED)

---

## Data collection

In this part, we will generate random user IDs and collect their respective user information if they exist.

### UID generation

Let's create random UIDs in the 0-3 billion range as discussed in the abstract.

We reshape them to simplify queries due to Twitter's API's rate limits.

If they have already been generated, we load them.

In [4]:
if CREATE_NEW_UIDs:
    uids = pd.DataFrame(
        np.array(
            random.sample(
                range(LOWER_ID_N, UPPER_ID_N), N_UID_PER_REQUEST * N_UID_REQUESTS
            )
        ).reshape(N_UID_PER_REQUEST, N_UID_REQUESTS)
    )
    uids.to_csv(UIDS_FILE, index=False)
else:
    uids = pd.read_csv(UIDS_FILE)

### Token load

To query Twitter's API, we need a bearer token which we load.

In [5]:
# Load bearer token
with open(BEARER_TOKEN, "r") as file:
    token = file.readline().strip("\n")

# Define authentication header
headers = {"Authorization": "Bearer " + token}

### User data collection

In this section, we will get user data from Twitter's API.

First we define a few helper functions.

In [6]:
def wait_for_reset(r):
    print(
        "Current time: {} (UTC)".format(datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"))
    )
    # Get reset time (Unix format)
    ts = int(r.headers["x-rate-limit-reset"])
    ts_str = datetime.fromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    # Compute difference between current time and reset time
    sleep_time = (datetime.fromtimestamp(ts) - datetime.utcnow()).total_seconds()
    if sleep_time > 0:
        print("Waiting until {} (UTC) for the rate limit to reset.".format(ts_str))
        time.sleep(sleep_time)
    else:
        print("Reset time was: {} (UTC)".format(ts_str))
    print("Resuming user data collection")


def get_user_data(req, headers=headers, wait=True):
    # Query the user data
    r = requests.get(req, headers=headers)
    if wait & (int(r.headers["x-rate-limit-remaining"]) == 0):
        wait_for_reset(r)

        # Query the user data
        r = requests.get(req, headers=headers)

    return r


def get_user_data_df(r):
    df = pd.DataFrame(
        r.json()["data"],
        columns=[
            "id",
            "username",
            "name",
            "protected",
            "withheld",
            "verified",
            "created_at",
            "location",
            "public_metrics",
            "description",
            "url",
            "entities",
            "pinned_tweet_id",
        ],
        # Replace NaNs by empty strings to facilitate pre-processing
    ).fillna("")
    return df

Now, let's query the API.

In [7]:
# Get user data
if COLLECT_USER_DATA:
    for i in trange(N_UID_PER_REQUEST // 100):
        # Get 100 UserIDs (limit per request as defined by Twitter)
        users = uids.values[i * 100 : (i + 1) * 100, REQUEST_NUMBER]
        # Define the request URL
        req = (
            API_USERS_ENDPOINT
            + ",".join([str(user) for user in users])
            + "&"
            + API_USER_FIELDS
            + "&"
            + API_TWEET_FIELDS
        )

        # Create the dataframe on the first iteration
        if i == 0:
            # Query the user data
            r = get_user_data(req)

            # If the rate limit is not maximal, then wait for the reset to occur
            # (max 15 minutes)
            if int(r.headers["x-rate-limit-remaining"]) != 299:
                wait_for_reset(r)

                # Query the user data
                r = get_user_data(req)

            raw_user_data = get_user_data_df(r)
        # Append to existing dataframe on other iterations
        # but do not wait for reset for the last iteration
        elif i == 299:
            # Query the user data
            r = get_user_data(req, wait=False)
            # Append new data to existing dataframe
            raw_user_data = raw_user_data.append(get_user_data_df(r))
        else:
            # Query the user data
            r = get_user_data(req)
            # Append new data to existing dataframe
            raw_user_data = raw_user_data.append(get_user_data_df(r))

There are a few important data points we will need to the next parts so we extract them here along with any others they are grouped with.

In [8]:
# Preprocess the data


def get_key_val(x, key):
    """Get dictionary value from key if it exists, otherwise return an empty string."""
    if key in x:
        return x[key]
    else:
        return ""


def get_public_metrics(df):
    """Extract the data from the public_metrics column"""
    for metric in ["followers_count", "following_count", "tweet_count", "listed_count"]:
        df[metric] = df.public_metrics.apply(lambda x: get_key_val(x, metric))
    df.pop("public_metrics")
    return df


def get_entities(df):
    """Extract the data from the entity column"""
    for entity in ["url", "description"]:
        df["entities_" + entity] = df.entities.apply(lambda x: get_key_val(x, entity))
    df.pop("entities")
    return df


if COLLECT_USER_DATA:
    raw_user_data = get_public_metrics(raw_user_data)
    raw_user_data = get_entities(raw_user_data)
    raw_user_data = raw_user_data.astype(
        {"id": int, "protected": bool, "verified": bool}
    )
    print("Number of valid users: {:,}".format(raw_user_data.shape[0]))
    raw_user_data

We need all of the user data available for the next parts, so we append the generated data (if any) to pre-existing user data and we save the data frame.

In [9]:
# Load user data if it exists
if os.path.isfile(USERS_FILE) and not FIRST_RUN:
    user_data = pd.read_csv(
        USERS_FILE,
        dtype={"id": int, "protected": bool, "verified": bool},
        lineterminator="\n",
    )
    if COLLECT_USER_DATA:
        user_data = user_data.append(raw_user_data)
else:
    user_data = raw_user_data

if COLLECT_USER_DATA:
    # Save data to disk
    user_data.to_csv(USERS_FILE, index=False)

# Print statistics
print("Total number of valid users: {:,}".format(user_data.shape[0]))

Total number of valid users: 33,520


### User subset definition

We define and save groups of users to facilitate data manipulation later on.

In [10]:
# Extract user subsets
if CREATE_USER_SUBSETS:
    # Public users
    public_users = user_data[~user_data.protected].copy()
    # Public users with tweets
    # Tweet count includes retweets
    public_users_w_tweets = user_data[
        ~user_data.protected & (user_data.tweet_count > 0)
    ].copy()
    # Public users with followers
    public_users_w_followers = user_data[
        ~user_data.protected & (user_data.followers_count > 0)
    ].copy()
    # Public users with friends
    public_users_w_friends = user_data[
        ~user_data.protected & (user_data.following_count > 0)
    ].copy()

    print("Number of public users: {:,}".format(public_users.shape[0]))
    print(
        "Number of public users with tweets: {:,}".format(
            public_users_w_tweets.shape[0]
        )
    )
    print(
        "Number of public users with followers: {:,}".format(
            public_users_w_followers.shape[0]
        )
    )
    print(
        "Number of public users with friends: {:,}".format(
            public_users_w_friends.shape[0]
        )
    )

    public_users.to_csv(PUBLIC_USERS_FILE, index=False)
    public_users_w_tweets.to_csv(PUBLIC_USERS_W_TWEETS_FILE, index=False)
    public_users_w_followers.to_csv(PUBLIC_USERS_W_FOLLOWERS_FILE, index=False)
    public_users_w_friends.to_csv(PUBLIC_USERS_W_FRIENDS_FILE, index=False)

else:
    public_users = pd.read_csv(
        PUBLIC_USERS_FILE,
        dtype={"id": int, "protected": bool, "verified": bool},
        lineterminator="\n",
    )
    public_users_w_tweets = pd.read_csv(
        PUBLIC_USERS_W_TWEETS_FILE,
        dtype={"id": int, "protected": bool, "verified": bool},
        lineterminator="\n",
    )
    public_users_w_followers = pd.read_csv(
        PUBLIC_USERS_W_FOLLOWERS_FILE,
        dtype={"id": int, "protected": bool, "verified": bool},
        lineterminator="\n",
    )
    public_users_w_friends = pd.read_csv(
        PUBLIC_USERS_W_FRIENDS_FILE,
        dtype={"id": int, "protected": bool, "verified": bool},
        lineterminator="\n",
    )

Number of public users: 31,239
Number of public users with tweets: 17,406
Number of public users with followers: 18,689
Number of public users with friends: 22,099


### Data pull status generation

As there are many queries to make, we create here a dataframe to be able to keep track of what data was already pulled and what data still needs to be pulled.

In [11]:
# Create pull status dataframe
if CREATE_DATA_PULL_STATUS:
    # Use public metrics to define limits
    user_data_pull_status = public_users[
        ["id", "followers_count", "following_count", "tweet_count"]
    ].copy()

    # Define parameters for API queries
    user_data_pull_status["timeline_lowest_id"] = 0
    user_data_pull_status["timeline_tweets_pulled"] = 0

    user_data_pull_status["followers_cursor"] = -1
    user_data_pull_status["followers_pulled"] = 0

    user_data_pull_status["following_cursor"] = -1
    user_data_pull_status["following_pulled"] = 0

    # Change column order for easier visualization
    user_data_pull_status = user_data_pull_status[
        [
            "id",
            "timeline_lowest_id",
            "timeline_tweets_pulled",
            "tweet_count",
            "followers_cursor",
            "followers_pulled",
            "followers_count",
            "following_cursor",
            "following_pulled",
            "following_count",
        ]
    ]

    # Set id column to index
    user_data_pull_status = user_data_pull_status.set_index("id")

else:
    user_data_pull_status = pd.read_csv(
        PUBLIC_USERS_PULL_STATUS_FILE,
        dtype=int,
        index_col="id",
        lineterminator="\n",
    )

### User timeline collection

As part of our analysis, we need to collect users' timelines. This is what we do here.

First we define a few helper functions whose names are pretty explicit, then we move on to actually query the data before saving it along with the data pull status.

In [12]:
def get_user_timeline_rate_limit():
    r = requests.get(API_V1_RATE_LIMITS, headers=headers)
    remaining = r.json()["resources"]["statuses"]["/statuses/user_timeline"][
        "remaining"
    ]
    # Get Unix timestamp
    reset_ts = r.json()["resources"]["statuses"]["/statuses/user_timeline"]["reset"]
    # Convert to string
    reset_time = datetime.fromtimestamp(reset_ts).strftime("%Y-%m-%d %H:%M:%S")
    return remaining, reset_time


def get_initial_timeline_df():
    return pd.DataFrame(
        columns=[
            "user_id",
            "user",
            "id",
            "created_at",
            "text",
            "in_reply_to_status_id",
            "in_reply_to_user_id",
            "source",
            "truncated",
            "coordinates",
            "place",
            "is_quote_status",
            "quoted_status_id",
            "quoted_status",
            "quote_count",
            "retweeted_status",
            "retweet_count",
            "favorite_count",
            "entities",
            "extended_entities",
            "possibly_sensitive",
            "lang",
        ]
    )


def get_user_timeline_df(r):
    df = pd.DataFrame(
        r.json(),
        columns=[
            "user_id",
            "user",
            "id",
            "created_at",
            "text",
            "in_reply_to_status_id",
            "in_reply_to_user_id",
            "source",
            "truncated",
            "coordinates",
            "place",
            "is_quote_status",
            "quoted_status_id",
            "quoted_status",
            "quote_count",
            "retweeted_status",
            "retweet_count",
            "favorite_count",
            "entities",
            "extended_entities",
            "possibly_sensitive",
            "lang",
        ],
        # Replace NaNs by empty strings to facilitate pre-processing
    ).fillna("")
    # Fill in user_id with tweet UserID
    df.user_id = df.user.apply(lambda x: x["id"])
    return df


def get_user_timeline(query_n, user_id, max_id, count):
    req = API_USER_TIMELINE_ENDPOINT
    params = {
        "user_id": str(user_id),
        "count": str(count),
        "include_rts": "1",
    }
    if max_id > 0:
        params.update({"max_id": str(max_id - 1)})

    r = requests.get(req, headers=headers, params=params)
    df = get_user_timeline_df(r)

    n_tweets_pulled = len(r.json())
    if n_tweets_pulled < count:
        print(r.url)
        print(
            "Query {:,} -- ".format(query_n + 1).ljust(15)
            + "User {}: got {:,} tweets instead of {:,}.".format(
                user_id, n_tweets_pulled, count
            )
        )
        lowest_id = -1
    if df.shape[0] > 0:
        lowest_id = int(df.id.min())

    return df, lowest_id, n_tweets_pulled

Having defined our helper functions, we now create or load our user timeline data and query the API for as much data as possible until we hit the rate limit (similar sections are run multiple times (days...) to query all of the necessary data).

In [13]:
# Load or create user timelines dataframe
if os.path.isfile(PUBLIC_USERS_TIMELINES_FILE) and not CREATE_DATA_PULL_STATUS:
    user_timeline_data = pd.read_csv(
        PUBLIC_USERS_TIMELINES_FILE,
        dtype={"coordinates": "object", "place": "object", "quoted_status": "object"},
        lineterminator="\n",
    )
else:
    user_timeline_data = get_initial_timeline_df()


if PULL_NEW_TIMELINE_DATA:

    # Get the number of available queries and rate limit reset time
    query_quota, reset_time = get_user_timeline_rate_limit()

    # Get users with tweets left to pull
    user_timelines_to_pull = user_data_pull_status[
        (user_data_pull_status.tweet_count > 0)
        & (
            user_data_pull_status.timeline_tweets_pulled
            < user_data_pull_status.tweet_count
        )
    ]

    n_queries = min(query_quota, user_timelines_to_pull.shape[0])
    print("Executing {:,} queries.".format(n_queries))
    for query_n in trange(n_queries):

        # Get query parameters
        user_id = user_timelines_to_pull.index[query_n]
        max_id = user_timelines_to_pull.loc[user_id, "timeline_lowest_id"]
        # A 200-tweet limit is set by Twitter per request
        count = min(
            user_timelines_to_pull.loc[user_id, "tweet_count"]
            - user_timelines_to_pull.loc[user_id, "timeline_tweets_pulled"],
            200,
        )

        # Get user timeline data and statistics
        raw_user_timeline_data, lowest_id, n_tweets_pulled = get_user_timeline(
            query_n, user_id, max_id, count
        )

        # Append to existing user timeline data
        user_timeline_data = user_timeline_data.append(raw_user_timeline_data)

        # Update pull status
        user_data_pull_status.loc[user_id, "timeline_lowest_id"] = lowest_id
        if user_timelines_to_pull.loc[user_id, "timeline_tweets_pulled"] == 0:
            user_data_pull_status.loc[user_id, "timeline_tweets_pulled"] = count
        else:
            user_data_pull_status.loc[user_id, "timeline_tweets_pulled"] += count

    print("Next reset time: {} (UTC)".format(reset_time))

Executing 1,500 queries.


https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=1427008184&count=155&include_rts=1
Query 5 --     User 1427008184: got 154 tweets instead of 155.
https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=124512329&count=200&include_rts=1
Query 18 --    User 124512329: got 199 tweets instead of 200.
https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=2872644659&count=63&include_rts=1
Query 26 --    User 2872644659: got 61 tweets instead of 63.
https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=970741922&count=65&include_rts=1
Query 27 --    User 970741922: got 61 tweets instead of 65.
https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=2942180530&count=5&include_rts=1
Query 31 --    User 2942180530: got 1 tweets instead of 5.
https://api.twitter.com/1.1/statuses/user_timeline.json?user_id=1411517634&count=200&include_rts=1
Query 40 --    User 1411517634: got 183 tweets instead of 200.
https://api.twitter.com/1.1/statuses/user_ti

We are interested in obtaining some statistics on the pulled data.

For consistency, we check whether the number of collected tweets is unique. As a measure of progress in our data collection efforts, we also report how many unique user timelines have been queried. As we cycle through users however, we will have queried parts of timelines for all users before we have queried all tweets from their timelines.

In [14]:
print(
    "Number of collected tweets: {:,} ({:,} unique) out of {:,}.\nNumber of unique users: {:,} (out of {:,}).".format(
        user_timeline_data.shape[0],
        len(np.unique(user_timeline_data.id.values)),
        np.sum(user_data_pull_status.tweet_count.values),
        len(np.unique(user_timeline_data.user_id.values)),
        public_users_w_tweets.shape[0],
    )
)

Number of collected tweets: 76,517 (76,517 unique) out of 17,538,830.
Number of unique users: 1,495 (out of 17,406).


To be able to use and share this data, we save it to disk.

In [15]:
if SAVE_NEW_TIMELINE_DATA:
    # Save data to disk
    user_timeline_data.to_csv(PUBLIC_USERS_TIMELINES_FILE, index=False)

if SAVE_PULLED_DATA:
    # Save data to disk
    user_data_pull_status.to_csv(PUBLIC_USERS_PULL_STATUS_FILE)

### Follower IDs collection

To build our ego network, we need to collect users' followers' IDs.

First we define a few helper functions, then we move on to actually query the data before saving it along with the data pull status.

In [16]:
def get_followers_ids_rate_limit():
    r = requests.get(API_V1_RATE_LIMITS, headers=headers)
    remaining = r.json()["resources"]["followers"]["/followers/ids"]["remaining"]
    # Get Unix timestamp
    reset_ts = r.json()["resources"]["followers"]["/followers/ids"]["reset"]
    # Convert to string
    reset_time = datetime.fromtimestamp(reset_ts).strftime("%Y-%m-%d %H:%M:%S")
    return remaining, reset_time


def get_initial_followers_df():
    return pd.DataFrame(
        columns=[
            "user_id",
            "ids",
            "next_cursor",
        ]
    )


def get_followers_df(r, user_id):
    df = pd.DataFrame(
        r.json(),
        columns=[
            "user_id",
            "ids",
            "next_cursor",
        ],
    ).fillna("")
    # Fill in user_id
    df.user_id = user_id
    return df


def get_followers(query_n, user_id, cursor, count):
    req = API_FOLLOWERS_IDS_ENDPOINT
    params = {
        "user_id": str(user_id),
        "cursor": str(cursor),
        "count": str(count),
    }

    r = requests.get(req, headers=headers, params=params)
    df = get_followers_df(r, user_id)

    n_followers_pulled = df.shape[0]
    if n_followers_pulled < count:
        print(r.url)
        print(
            "Query {:,} -- ".format(query_n + 1).ljust(15)
            + "User {}: got {:,} followers instead of {:,}.".format(
                user_id, n_followers_pulled, count
            )
        )
        next_cursor = -1
    if df.shape[0] > 0:
        next_cursor = df.next_cursor[0]

    return df, next_cursor

Having defined our helper functions, we now create or load our users' followers' data and query the API for as much data as possible until we hit the rate limit.

In [17]:
# Load or create user followers dataframe
if os.path.isfile(PUBLIC_USERS_FOLLOWERS_FILE) and not CREATE_DATA_PULL_STATUS:
    user_followers_data = pd.read_csv(
        PUBLIC_USERS_FOLLOWERS_FILE,
        dtype=int,
        lineterminator="\n",
    )
else:
    user_followers_data = get_initial_followers_df()


if PULL_NEW_FOLLOWER_DATA:

    # Get the number of available queries and rate limit reset time
    query_quota, reset_time = get_followers_ids_rate_limit()

    # Get users with tweets left to pull
    user_followers_to_pull = user_data_pull_status[
        (user_data_pull_status.followers_count > 0)
        & (
            user_data_pull_status.followers_pulled
            < user_data_pull_status.followers_count
        )
    ]

    n_queries = min(query_quota, user_followers_to_pull.shape[0])
    print("Executing {:,} queries.".format(n_queries))
    for query_n in trange(n_queries):

        # Get query parameters
        user_id = user_followers_to_pull.index[query_n]
        cursor = user_followers_to_pull.loc[user_id, "followers_cursor"]
        # A 5000-UserIDs limit is set by Twitter per request
        count = min(
            user_followers_to_pull.loc[user_id, "followers_count"]
            - user_followers_to_pull.loc[user_id, "followers_pulled"],
            5000,
        )

        # Get user followers data and statistics
        raw_user_followers_data, next_cursor = get_followers(
            query_n, user_id, cursor, count
        )

        # Append to existing user followers data
        user_followers_data = user_followers_data.append(raw_user_followers_data)

        # Update pull status
        user_data_pull_status.loc[user_id, "followers_cursor"] = next_cursor
        if user_followers_to_pull.loc[user_id, "followers_pulled"] == 0:
            user_data_pull_status.loc[user_id, "followers_pulled"] = count
        else:
            user_data_pull_status.loc[user_id, "followers_pulled"] += count

    print("Next reset time: {} (UTC)".format(reset_time))

Executing 15 queries.


https://api.twitter.com/1.1/followers/ids.json?user_id=555533734&cursor=-1&count=311
Query 4 --     User 555533734: got 310 followers instead of 311.

Next reset time: 2020-12-08 17:16:26 (UTC)


Once again, we are interested in obtaining some statistics on the pulled data.

As a measure of progress in our data collection efforts, we report how many followers we have collected and how many unique users have been queried. As we cycle through users however, we will have queried all users before we have queried all of their follower lists.

In [18]:
print(
    "Number of collected followers: {:,} (out of {:,}).\nNumber of unique users: {:,} (out of {:,}).".format(
        user_followers_data.shape[0],
        np.sum(user_data_pull_status.followers_count.values),
        len(np.unique(user_followers_data.user_id.values)),
        public_users_w_followers.shape[0],
    )
)

Number of collected followers: 419 (out of 1,662,736).
Number of unique users: 15 (out of 18,689).


Once again, we save the data to disk.

In [19]:
if SAVE_NEW_FOLLOWER_DATA:
    # Save data to disk
    user_followers_data.to_csv(PUBLIC_USERS_FOLLOWERS_FILE, index=False)

In [20]:
if SAVE_PULLED_DATA:
    # Save data to disk
    user_data_pull_status.to_csv(PUBLIC_USERS_PULL_STATUS_FILE)

### Friend IDs collection

To build our ego network, we also need to collect users' friends' IDs.

First we define a few helper functions, then we move on to actually query the data before saving it along with the data pull status as usual.

In [21]:
def get_friends_ids_rate_limit():
    r = requests.get(API_V1_RATE_LIMITS, headers=headers)
    remaining = r.json()["resources"]["friends"]["/friends/ids"]["remaining"]
    # Get Unix timestamp
    reset_ts = r.json()["resources"]["friends"]["/friends/ids"]["reset"]
    # Convert to string
    reset_time = datetime.fromtimestamp(reset_ts).strftime("%Y-%m-%d %H:%M:%S")
    return remaining, reset_time


def get_initial_friends_df():
    return pd.DataFrame(
        columns=[
            "user_id",
            "ids",
            "next_cursor",
        ]
    )


def get_friends_df(r, user_id):
    df = pd.DataFrame(
        r.json(),
        columns=[
            "user_id",
            "ids",
            "next_cursor",
        ],
    ).fillna("")
    # Fill in user_id
    df.user_id = user_id
    return df


def get_friends(query_n, user_id, cursor, count):
    req = API_FRIENDS_IDS_ENDPOINT
    params = {
        "user_id": str(user_id),
        "cursor": str(cursor),
        "count": str(count),
    }

    r = requests.get(req, headers=headers, params=params)
    df = get_friends_df(r, user_id)

    n_friends_pulled = df.shape[0]
    if n_friends_pulled < count:
        print(r.url)
        print(
            "Query {:,} -- ".format(query_n + 1).ljust(15)
            + "User {}: got {:,} friends instead of {:,}.".format(
                user_id, n_friends_pulled, count
            )
        )
        next_cursor = -1
    if df.shape[0] > 0:
        next_cursor = df.next_cursor[0]

    return df, next_cursor

Having defined our helper functions, we now create or load our users' friends' data and query the API for as much data as possible until we hit the rate limit.

In [22]:
# Load or create user friends dataframe
if os.path.isfile(PUBLIC_USERS_FRIENDS_FILE) and not CREATE_DATA_PULL_STATUS:
    user_friends_data = pd.read_csv(
        PUBLIC_USERS_FRIENDS_FILE,
        dtype=int,
        lineterminator="\n",
    )
else:
    user_friends_data = get_initial_friends_df()


if PULL_NEW_FRIEND_DATA:

    # Get the number of available queries and rate limit reset time
    query_quota, reset_time = get_friends_ids_rate_limit()

    # Get users with tweets left to pull
    user_friends_to_pull = user_data_pull_status[
        (user_data_pull_status.following_count > 0)
        & (
            user_data_pull_status.following_pulled
            < user_data_pull_status.following_count
        )
    ]

    n_queries = min(query_quota, user_friends_to_pull.shape[0])
    print("Executing {:,} queries.".format(n_queries))
    for query_n in trange(n_queries):

        # Get query parameters
        user_id = user_friends_to_pull.index[query_n]
        cursor = user_friends_to_pull.loc[user_id, "following_cursor"]
        # A 5000-UserIDs limit is set by Twitter per request
        count = min(
            user_friends_to_pull.loc[user_id, "following_count"]
            - user_friends_to_pull.loc[user_id, "following_pulled"],
            5000,
        )

        # Get user friends data and statistics
        raw_user_friends_data, next_cursor = get_friends(
            query_n, user_id, cursor, count
        )

        # Append to existing user friends data
        user_friends_data = user_friends_data.append(raw_user_friends_data)

        # Update pull status
        user_data_pull_status.loc[user_id, "following_cursor"] = next_cursor
        if user_friends_to_pull.loc[user_id, "following_pulled"] == 0:
            user_data_pull_status.loc[user_id, "following_pulled"] = count
        else:
            user_data_pull_status.loc[user_id, "following_pulled"] += count

    print("Next reset time: {} (UTC)".format(reset_time))

Executing 15 queries.



Next reset time: 2020-12-08 17:16:30 (UTC)


Much like the previous section, we are interested in seeing some statistics on the collected data.

In [23]:
print(
    "Number of collected friends: {:,} (out of {:,}).\nNumber of unique users: {:,} (out of {:,}).".format(
        user_friends_data.shape[0],
        np.sum(user_data_pull_status.following_count.values),
        len(np.unique(user_friends_data.user_id.values)),
        public_users_w_friends.shape[0],
    )
)

Number of collected friends: 378 (out of 2,208,697).
Number of unique users: 15 (out of 22,099).


Once again, we save the data to disk.

In [24]:
if SAVE_NEW_FRIEND_DATA:
    # Save data to disk
    user_friends_data.to_csv(PUBLIC_USERS_FRIENDS_FILE, index=False)

In [25]:
if SAVE_PULLED_DATA:
    # Save data to disk
    user_data_pull_status.to_csv(PUBLIC_USERS_PULL_STATUS_FILE)

---

# Break

In [26]:
break

SyntaxError: 'break' outside loop (<ipython-input-26-6aaf1f276005>, line 1)

# Test section

In [ ]:
user_data_pull_status[user_data_pull_status.tweet_count > 0].head(15)

In [ ]:
get_initial_followers_df().shape[0]

In [ ]:
r = requests.get(
    "https://api.twitter.com/1.1/followers/ids.json?user_id=555533734&cursor=-1&count=311",
    headers=headers,
)

In [ ]:
test = pd.DataFrame(
    r.json(),
    columns=[
        "user_id",
        "ids",
        "next_cursor",
    ],
    dtype=int,
).fillna("")
test

In [ ]:
len(r.json()["ids"])

In [ ]:
np.unique(user_timeline_data.id.values).shape[0]

In [ ]:
import os

os.path.isfile(PUBLIC_USERS_FILE)

In [ ]:
pd.DataFrame(
    columns=[
        "user_id",
        "user",
        "id",
        "created_at",
        "text",
        "in_reply_to_status_id",
        "in_reply_to_user_id",
        "source",
        "truncated",
        "coordinates",
        "place",
        "is_quote_status",
        "quoted_status_id",
        "quoted_status",
        "quote_count",
        "retweeted_status",
        "retweet_count",
        "favorite_count",
        "entities",
        "extended_entities",
        "possibly_sensitive",
        "lang",
    ],
    # Replace NaNs by empty strings to facilitate pre-processing
)

In [ ]:
user_data_pull_status = user_data[
    ["id", "followers_count", "following_count", "tweet_count"]
].copy()

In [ ]:
user_data_pull_status["timeline_lowest_id"] = -1
user_data_pull_status["timeline_tweets_pulled"] = -1

user_data_pull_status["followers_cursor"] = -1
user_data_pull_status["followers_pulled"] = -1

user_data_pull_status["following_cursor"] = -1
user_data_pull_status["following_pulled"] = -1

In [ ]:
user_data_pull_status = user_data_pull_status[
    [
        "id",
        "timeline_lowest_id",
        "timeline_tweets_pulled",
        "tweet_count",
        "followers_cursor",
        "followers_pulled",
        "followers_count",
        "following_cursor",
        "following_pulled",
        "following_count",
    ]
]
user_data_pull_status

In [ ]:
req = (
    "https://api.twitter.com/1.1/application/rate_limit_status.json"
    #     + "783214,15994119,1320117356"
    #     + "&"
    #     + API_USER_FIELDS
    #     + "&"
    #     + API_TWEET_FIELDS
)
payload = {"resources": "application,statuses,followers,friends"}
print(req)
r = requests.get(req, headers=headers, params=payload)
print(r.url)

In [ ]:
r.json()["resources"]["statuses"]["/statuses/user_timeline"]

In [ ]:
req = API_USER_TIMELINE_ENDPOINT + "?user_id=783214" + "&count=10"
print(req)
r = requests.get(req, headers=headers)

In [ ]:
r.json()

In [ ]:
test = get_user_timeline_df(r)

In [ ]:
test

In [ ]:
test.created_at = pd.to_datetime(test.created_at)

In [ ]:
test.sort_values(by="id").user[0]

In [ ]:
int(r.headers["x-rate-limit-remaining"])

In [ ]:
wait_for_reset(r)

In [ ]:
df = pd.DataFrame(
    r.json()["data"],
    columns=[
        "id",
        "username",
        "name",
        "protected",
        "withheld",
        "verified",
        "created_at",
        "location",
        "public_metrics",
        "description",
        "url",
        "entities",
        "pinned_tweet_id",
    ],
).fillna("")
df

In [ ]:
df.public_metrics[0]

In [ ]:
783214 in user_data.id.values.astype(int)